<a href="https://colab.research.google.com/github/TanmayKumar-EngStud/GenderClassification/blob/main/GenderClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Python code for loading and refining Data

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

DataDir = "/content/drive/MyDrive/newSET"
Category = ["women","men"]
print('done')

done


In [12]:
trainingData = []
IMG_SIZE = 250
def create_Training_Data():
    for cat in Category:
        path = os.path.join(DataDir, cat)
        class_num= Category.index(cat)
        for img in os.listdir(path):
            try:
                img_arr   = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
                trainingData.append([new_array, class_num])
            except Exception as e:
                pass
create_Training_Data()            
print(len(trainingData))


3262


In [13]:
import random 
random.shuffle(trainingData)
X =[]
y =[]

for features, label in trainingData:
    X.append(features)
    y.append(label)


X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)
print('done')

done


In [14]:
import pickle 

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

X = X/255.0
print('done')

done


## **Creating the model** 


```
# This the actual code of my Convolution network
```


---





In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
import time

## Once Again calling the X and y value to make it a numpy array

In [16]:
X = np.array(X)
y =np.array(y)
print(X.shape[1:])

(250, 250, 1)


## Building the model
---


> Defining the function which creates model




In [17]:
def createModel(DenseLayer, convLayer,LayerSize):

    model= Sequential()
    model.add(Conv2D(LayerSize,(3,3),input_shape= (250,250,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size= (2,2)))
    
    for i in range (convLayer -1):
      model.add(Conv2D(LayerSize,(3,3)))
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size= (2,2)))


    model.add(Flatten())
    for i in range (DenseLayer-1):
      model.add(Dense(64))
      model.add(Activation('relu'))


    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

# I have compared the model designs with one another
---


*   **Conisdered 4 sets of Dense Layers** (3,4) 
*   **Considered 3 sets of Neurons in Each layer**  (32,64)
*   **Considered 3 sets of convolutional layers**  (2,3) 
***
Overall 4x3x3 = 36 Models will be made


In [19]:

!mkdir -p /content/drive/MyDrive/newSET/saved_model


In [ ]:
denseLayers = [3,4]
layerSizes = [32, 64]
convLayers = [2,3]

for denseLayer in denseLayers:
  for layerSize in layerSizes:
    for convLayer in convLayers:
      NAME = 'men-x-women-CNN-{}x2-{} -convLayers-{} -denseLayer-{}'.format(layerSize,int(time.time()),convLayer, denseLayer)
      tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/newSET/logs/{}'.format(NAME))
      model = createModel(denseLayer, convLayer,layerSize)
      model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])
      C_path = '/content/drive/MyDrive/newSET/saved_model/training_{}x2_convLayers-{}-denseLayer-{}/cp.ckpt'.format(layerSize,convLayer, denseLayer)
      ck_dir = os.path.dirname(C_path)
      callback = ModelCheckpoint(C_path, save_weights_only = True, verbose = 1)
      model.fit(X,y, batch_size = 16, validation_split=0.1, epochs = 5, callbacks = [tensorboard, callback])
      


#**Saving the model**


---
by using the technique of checkpoint callback 



## After Comparision we can clearly say that:-
>'men-x-women-CNN-64x2-convLayers-2-denselayer-3'
>
>'/content/drive/MyDrive/newSET/saved_model/training_64x2convLayers-2-denseLayer-3'


is the best as compared to others

Now we will use this Model to further validate our dataset

In [35]:
model = createModel(3,2,64)

model.load_weights('/content/drive/MyDrive/newSET/saved_model/training_64x2_convLayers-2-denseLayer-3/cp.ckpt')


# Reloading the previous model
---
Using that model and testing

In [64]:
import cv2
import tensorflow as tf


Categories = ["female","male"]
def prepare(filepath):
  IMG_SIZE = 250
  img_arr   = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
  new_image = cv2.resize(img_arr, (IMG_SIZE,IMG_SIZE))
  return new_image.reshape(-1,IMG_SIZE,IMG_SIZE,1)

prediction = model.predict(prepare("/content/drive/MyDrive/GenderTesting/test(ss).jpg"))


print(Categories[int(prediction[0][0])])

male
